In [ ]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np

In [ ]:
def img_crop (image):
  height=image.shape[0]
  width=image.shape[1]
  h_lower=int(height/2+ (0.45*height))
  h_upper=int(height/2- (0.38*height))
  w_left=int(width/2-(0.3*width))
  w_right=int(width/2+(0.2*width))
  #print(h_upper)
  image = image[h_upper:h_lower, w_left:w_right, :]

  return image

In [ ]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=30):
    name = video_path.split("/")[-1].split(".")[0].lower()
    subname=name[19:]
    
  
  
    print(name)
    print(subname)
    if (name.find("_bhu") != -1) :
          save_path = os.path.join(save_dir, "Bhujangasana",subname)  
    elif (name.find("_padam") != -1) or (name.find("_padma") != -1):
          save_path = os.path.join(save_dir, "Padamasana",subname)
    elif (name.find("_shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana",subname)
    elif (name.find("_trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana",subname)
    elif (name.find("_vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana",subname)
    elif (name.find("_tad") != -1):
        save_path = os.path.join(save_dir, "Tadasana",subname)
    else:
        print(subname)
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            image=img_crop(frame)
            cv2.imwrite(f"{save_path}/{idx}.png", image)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("Yoga_Vid_Collected/*")
    save_dir = "dataset"

    for path in video_paths:
        save_frame(path, save_dir, gap=30)

In [ ]:
# Load the input image.
def image_to_movenet(path,label,keypts_label):
    image_path = path
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)

    # Download the model from TF Hub.
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    movenet = model.signatures['serving_default']

    # Run model inference.
    outputs = movenet(image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints = outputs['output_0']
    keypoints_tup=(keypoints,label)
    keypts_label.append(keypoints_tup)
    

In [10]:
import os
addresses=[]
keypts_labels=[]
class_list=os.listdir('dataset')
for name in class_list:
    for image in os.listdir(f"dataset/{name}"):
        for item in os.listdir(f"dataset/{name}/{image}"):
            addresses.append(os.path.abspath(f"dataset/{name}/{image}/{item}"))
#             image_to_movenet(os.path.abspath(f"dataset/{name}/{image}/{item}"),name,keypts_labels)
# print(keypts_labels[500])


['C:\\Users\\radha\\OneDrive\\Desktop\\BE\\Yoga_Class-Analysis\\dataset\\Bhujangasana\\abhay_bhuj\\105.png', 'C:\\Users\\radha\\OneDrive\\Desktop\\BE\\Yoga_Class-Analysis\\dataset\\Bhujangasana\\abhay_bhuj\\135.png', 'C:\\Users\\radha\\OneDrive\\Desktop\\BE\\Yoga_Class-Analysis\\dataset\\Bhujangasana\\abhay_bhuj\\15.png', 'C:\\Users\\radha\\OneDrive\\Desktop\\BE\\Yoga_Class-Analysis\\dataset\\Bhujangasana\\abhay_bhuj\\165.png', 'C:\\Users\\radha\\OneDrive\\Desktop\\BE\\Yoga_Class-Analysis\\dataset\\Bhujangasana\\abhay_bhuj\\195.png', 'C:\\Users\\radha\\OneDrive\\Desktop\\BE\\Yoga_Class-Analysis\\dataset\\Bhujangasana\\abhay_bhuj\\225.png', 'C:\\Users\\radha\\OneDrive\\Desktop\\BE\\Yoga_Class-Analysis\\dataset\\Bhujangasana\\abhay_bhuj\\255.png', 'C:\\Users\\radha\\OneDrive\\Desktop\\BE\\Yoga_Class-Analysis\\dataset\\Bhujangasana\\abhay_bhuj\\285.png', 'C:\\Users\\radha\\OneDrive\\Desktop\\BE\\Yoga_Class-Analysis\\dataset\\Bhujangasana\\abhay_bhuj\\315.png', 'C:\\Users\\radha\\OneDrive\